In [142]:
from __future__ import print_function, division
import cv2
import torch
import numpy as np
import pickle
import math

In [143]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
data = pickle.load(open("toy_example/data/toyexample_1_data.p", 'rb'))
img = cv2.imread("toy_example/data/toyexample_1.png")

In [145]:
T_perturbed = np.array([[math.cos(10*math.pi/180), -math.sin(10*math.pi/180), 0, 0],
             [math.sin(10*math.pi/180), math.cos(10*math.pi/180), 0, 0],
             [0, 0, 1, 0]])

In [146]:
data['coords'] = np.around(data['2d_points']).astype(int) - 1

In [147]:
P_perturbed = np.dot(data['K'], T_perturbed)
projected_2d = np.dot(P_perturbed, np.concatenate((data['3d_points'], np.ones(len(data['3d_points']))[:, None]),-1).T)
projected_2d = (projected_2d.T/projected_2d.T[:,2,None])[:, :2]

In [148]:
coords_2d = np.around(projected_2d)
coords_2d = coords_2d.astype(int) - 1

In [149]:
img = img.astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)

0 [112 128]
1 [102 138]
2 [70 68]
3 [115 119]


In [150]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [141]:
cv2.imwrite('perturbed_points.png', img)

True

### run BA

In [116]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model2 import sparse3DBA
from featureBA.src.utils import (from_homogeneous, to_homogeneous,
                batched_eye_like, skew_symmetric, so3exp_map, sobel_filter, squared_loss, scaled_loss)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('double')

In [118]:
img_torch = torch.from_numpy(img)[None,...]
grad_x, grad_y = sobel_filter(img_torch)

In [135]:
img = grad_y.numpy().reshape(img.shape).astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

0 [112 128]
1 [102 138]
2 [70 68]
3 [115 119]


In [120]:
cv2.imwrite("grad_x.png", grad_x.numpy().reshape(img.shape).astype('uint8'))
cv2.imwrite("grad_y.png", grad_y.numpy().reshape(img.shape).astype('uint8'))

True

In [121]:
pts3D = torch.from_numpy(data['3d_points'][:,:3])
ref2d = torch.from_numpy(np.flip(data['coords'], axis = 1).copy())
feature_ref = torch.cat([img_torch[:, i, j].unsqueeze(0) for i, j in zip(ref2d[:,0], ref2d[:,1])]).type(torch.DoubleTensor)
feature_map_query = img_torch.type(torch.DoubleTensor)
R_init, t_init = torch.from_numpy(T_perturbed[:, :3]), torch.from_numpy(T_perturbed[:, 3])
feature_grad_x = grad_x
feature_grad_y = grad_y
K = torch.from_numpy(data['K'])

In [122]:
from __future__ import print_function, division

from featureBA.src.utils import (from_homogeneous, to_homogeneous,
                                 batched_eye_like, skew_symmetric, so3exp_map)

from featureBA.src.utils import squared_loss, scaled_loss
import torch
from torch import nn
import numpy as np

def optimizer_step(g, H, lambda_=0):
    """One optimization step with Gauss-Newton or Levenberg-Marquardt.
    Args:
        g: batched gradient tensor of size (..., N).
        H: batched hessian tensor of size (..., N, N).
        lambda_: damping factor for LM (use GN if lambda_=0).
    """
    if lambda_:  # LM instead of GN
        D = (H.diagonal(dim1=-2, dim2=-1) + 1e-9).diag_embed()
        H = H + D*lambda_
    try:
        P = torch.inverse(H)
    except RuntimeError as e:
        logging.warning(f'Determinant: {torch.det(H)}')
        raise e
    delta = -(P @ g[..., None])[..., 0]
    return delta

def indexing_(feature_map, points):
    '''
    Function gives x and y gradients for 3D points in camera frame.

    inputs: (All pytorch tensors)
    @feature_map : x gradient of the feature map (CxHxW)
    @points : pixel coordinates of points (Nx2)

    outputs: 
    features : features for the points (NxC)
    '''

    features = torch.cat([feature_map[:, i, j].unsqueeze(0) for i, j in zip(points[:,0], points[:,1])])

    return features

In [18]:
n_iters = 100
loss_fn = squared_loss
lambda_ = 0.01

R = R_init
t = t_init

NameError: name 'squared_loss' is not defined

In [132]:
p_3d_1 = (torch.mm(R, pts3D.T).T + t)
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
print((error))

tensor([[-10.],
        [-26.],
        [-52.],
        [ -8.]], dtype=torch.float64)


In [133]:
error = torch.flip(p_proj_1,(1,)) - ref2d

In [134]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('uint8')
for i, p in enumerate(p_proj_1):
    print(i, p)
    cv2.circle(img, tuple(p.numpy()), 1, (128, 128, 0), -1)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('optimized_pos.png', img)

0 tensor([109, 129], dtype=torch.int32)
1 tensor([100, 139], dtype=torch.int32)
2 tensor([54, 73], dtype=torch.int32)
3 tensor([111, 120], dtype=torch.int32)


True

In [634]:
i = 0
verbose = 1
p_3d_1 = torch.mm(R, pts3D.T).T + t
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
# error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
error = torch.flip(p_proj_1, (1,)) - ref2d
error = error.type(torch.DoubleTensor)
print(error)
cost = 0.5*(error**2).sum(-1)

if i == 0:
    prev_cost = cost.mean(-1)
if verbose:
    print('Iter ', i, cost.mean().item())

tensor([[-15.,   6.]], dtype=torch.float64)
Iter  0 130.5


In [635]:
J_p_T = torch.cat([
    batched_eye_like(p_3d_1, 3), -skew_symmetric(p_3d_1)], -1)

In [636]:
shape = p_3d_1.shape[:-1]
o, z = p_3d_1.new_ones(shape), p_3d_1.new_zeros(shape)
J_e_p = torch.stack([
    K[0,0]*o, z, -K[0,0]*p_3d_1[..., 0] / p_3d_1[..., 2],
    z, K[1,1]*o, -K[1,1]*p_3d_1[..., 1] / p_3d_1[..., 2],
], dim=-1).reshape(shape+(2, 3)) / p_3d_1[..., 2, None, None]

In [637]:
grad_x_points = indexing_(feature_grad_x, torch.flip(p_proj_1,(1,)))
grad_y_points = indexing_(feature_grad_y, torch.flip(p_proj_1,(1,)))
J_p_F = torch.cat((grad_x_points[..., None], grad_y_points[...,None]), -1)

In [638]:
J_e_T = J_p_F @ J_e_p @ J_p_T

In [639]:
J_e_T = J_e_p @ J_p_T
print(J_e_T)

tensor([[[  1.3709,   0.0000,  -5.0881, -26.1725,  73.8806,  -7.0515],
         [  0.0000,   1.3709,  -1.9333, -14.9447,  26.1725,  18.5581]]],
       dtype=torch.float64)


In [640]:
Grad = torch.einsum('...ijk,...ij->...ik', J_e_T, error)
Grad = Grad.sum(-2)  # Grad was ... x N x 6

In [641]:
J = J_e_T
Hess = torch.einsum('...ijk,...ijl->...ikl', J, J)
# Hess = weights[..., None, None] * Hess
Hess = Hess.sum(-3)  # Hess was ... x N x 6 x 6

In [642]:
torch.matmul(torch.inverse(Hess), Grad[..., None])

tensor([[79.8540],
        [19.9733],
        [-2.2835],
        [ 3.3155],
        [-0.7431],
        [-0.4048]], dtype=torch.float64)

In [643]:
delta = -0.1*torch.matmul(torch.inverse(Hess), Grad[...,None])[..., 0]

In [644]:
dt, dw = delta[..., :3], delta[..., 3:6]
dr = so3exp_map(dw)
print("dr is : ", dw)
print("dt is : ", dt)
R_new = dr @ R
t_new = dr @ t + dt

dr is :  tensor([-0.3316,  0.0743,  0.0405], dtype=torch.float64)
dt is :  tensor([-7.9854, -1.9973,  0.2284], dtype=torch.float64)


In [630]:
print(" R_new ", R_new)
print(" t_new ", t_new)
print("R ", R)
print("t ", t)

 R_new  tensor([[ 0.8589,  0.5121, -0.0086],
        [-0.4722,  0.7984,  0.3737],
        [ 0.1982, -0.3169,  0.9275]], dtype=torch.float64)
 t_new  tensor([-6.3682, -2.3256,  0.6918], dtype=torch.float64)
R  tensor([[ 0.8271,  0.5574, -0.0721],
        [-0.5549,  0.8302,  0.0533],
        [ 0.0895, -0.0041,  0.9960]], dtype=torch.float64)
t  tensor([ 1.5656, -0.5440,  0.4368], dtype=torch.float64)


In [615]:
R, t = R_new, t_new

In [127]:
model = sparse3DBA(n_iters = 100, lambda_ = 0.01)

In [128]:
R, t = model(pts3D, ref2d, feature_map_query, feature_grad_x, feature_grad_y, K, R_init, t_init)

prev cost is  tensor(26., dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 0.0068,  0.0010, -0.1173], dtype=torch.float64)
dt is :  tensor([-2.2699, -0.7315,  0.0997], dtype=torch.float64)
new cost is  tensor(19.7500, dtype=torch.float64)
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 0.0128, -0.0013,  0.0386], dtype=torch.float64)
dt is :  tensor([ 0.4302,  2.7616, -0.0147], dtype=torch.float64)
new cost is  tensor(41.5000, dtype=torch.float64)
cost increased, continue with next iteration
0.01
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 0.0069, -0.0028,  0.0063], dtype=torch.float64)
dt is :  tensor([ 0.6164,  2.1770, -0.0395], dtype=torch.float64)
new cost is  tensor(35.8750, dtype=torch.float64)
cost increased, continue with next iteration
0.1
prev cost is  

dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
1

new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2

dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size([4, 3, 6])
J_e_p is  torch.Size([4, 2, 3])
dr is :  tensor([ 4.7223e-07, -3.2322e-07, -9.2191e-06], dtype=torch.float64)
dt is :  tensor([-7.3512e-05,  1.3238e-04, -5.3123e-06], dtype=torch.float64)
new cost is  tensor(21.3750, dtype=torch.float64)
cost increased, continue with next iteration
10000.0
prev cost is  tensor(19.7500, dtype=torch.float64)
J_p_T is  torch.Size

In [129]:
R

tensor([[ 9.9854e-01, -5.3988e-02,  4.6671e-04],
        [ 5.3990e-02,  9.9852e-01, -7.0458e-03],
        [-8.5632e-05,  7.0607e-03,  9.9998e-01]], dtype=torch.float64)

In [130]:
t

tensor([[ 0.9848, -0.1736,  0.0000],
        [ 0.1736,  0.9848,  0.0000],
        [ 0.0000,  0.0000,  1.0000]], dtype=torch.float64)

In [131]:
t

tensor([-2.2972, -0.6782,  0.0975], dtype=torch.float64)

In [349]:
a @ b

array([[[1.13206942, 1.11894092, 0.57274033, 1.24841041, 0.79335565,
         0.46561771],
        [1.2855546 , 1.19792658, 1.26153542, 1.5414339 , 1.17330447,
         1.04735982]]])